# Lesson 13: Going Deeper with Diffusion Models: Conditioning, Guidance and Latent Diffusion

## The Plan

- Load a diffusers pipeline and generate an image

- Sampling loop with a new sampler (plus DDIM explanation?)

- Fine-tune on a new dataset

- Guide with a loss or two (start simple, then CLIP)

- Condition on a class 

- Classifier-free guidance

- Latent Diffusion Model

- Progressive Distillation


